In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

from tqdm import tqdm_notebook

import matplotlib.pyplot as plt

from tqdm import tqdm_notebook

from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.neighbors import NearestNeighbors

%matplotlib inline
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier

from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor

In [2]:
##links = pd.read_csv('links.csv')
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
tags = pd.read_csv('tags.csv')

In [3]:
def change_string(s):
    return ' '.join(s.lower().replace(' ', '').replace('-', '').split('|'))

In [4]:
# Нормализуем жанры
movies['genres_formatted'] = movies.genres.apply(change_string)


In [5]:
movies=movies.drop(['genres'], axis=1)

In [6]:
# слепим теги каждого юзера,  если они относятся к одному фильму
tags['tag_new'] = tags[['userId','movieId','tag']].groupby(['userId','movieId'])['tag'].transform(lambda x: ' '.join(x))
tags=tags[['userId','movieId','tag_new']].drop_duplicates()

### Причешем наши теги

In [7]:
import pymorphy2
from nltk.tokenize import word_tokenize
import nltk
#nltk.download('punkt')
from tqdm import tqdm

In [8]:
class MorphProvider:
    def __init__(self):
        self.cache = {}
        self.morph = pymorphy2.MorphAnalyzer()
    
    def __call__(self, w):
        w = w.lower()
        cached = self.cache.get(w)
        if cached:
            return cached
        try:
            morphed = self.morph.parse(w)[0].normal_form
            self.cache[w] = morphed
            return morphed
        except:
            return None
    
    def morph_string(self, s):
        words = word_tokenize(s)
        return " ".join([self.__call__(w) for w in words])

In [9]:
morph = MorphProvider()

In [10]:
tags["tag_normalized"] = [morph.morph_string(t) for t in tqdm(tags.tag_new)]

100%|██████████| 1775/1775 [00:00<00:00, 8767.36it/s]


In [11]:
tags=tags.drop(['tag_new'], axis=1)

In [12]:
tags.head()

,userId,movieId,tag_normalized
0,2,60756,funny highly quotable will ferrell
3,2,89774,boxing story mma tom hardy
6,2,106782,drugs leonardo dicaprio martin scorsese
9,7,48516,way too long
10,18,431,al pacino gangster mafia


## Сделаем какой-то бейслайн простой:

### Посчитаем разные метрики

In [13]:
from scipy import stats

In [14]:
raitis_aggr= ratings.groupby(['movieId'])[['rating']].count()
raitis_aggr.columns=['count_']
raitis_aggr['mean_']= ratings.groupby(['movieId'])[['rating']].mean()
raitis_aggr['median_']= ratings.groupby(['movieId'])[['rating']].median()
raitis_aggr['var_']= ratings.groupby(['movieId'])[['rating']].var()
raitis_aggr['mode']=ratings.groupby(['movieId']).rating.agg(lambda x: stats.mode(x)[0])

In [15]:
raitis_aggr.shape

(9724, 5)

In [16]:
raitis_aggr.head(4)

,count_,mean_,median_,var_,mode
movieId,,,,,
1,215,3.920930,4.0,0.696990,4.0
2,110,3.431818,3.5,0.777419,4.0
3,52,3.259615,3.0,1.112651,3.0
4,7,2.357143,3.0,0.726190,3.0


In [17]:
ratings[['userId','movieId','rating']].shape

(100836, 3)

In [18]:
x = pd.merge(ratings[['userId','movieId','rating']], raitis_aggr, on='movieId', how='left')

In [19]:
y=x.rating


In [20]:
x=x.drop(['rating'], axis=1)
x=x.drop(['userId'], axis=1)
x=x.drop(['movieId'], axis=1)

In [21]:
x.head()

,count_,mean_,median_,var_,mode
0,215,3.920930,4.0,0.696990,4.0
1,52,3.259615,3.0,1.112651,3.0
2,102,3.946078,4.0,0.667856,4.0
3,203,3.975369,4.0,0.850875,4.0
4,204,4.237745,4.5,0.641475,5.0


In [22]:

x.isnull().any()

count_     False
mean_      False
median_    False
var_        True
mode       False
dtype: bool

In [23]:
#заменим  пропуски в дисперсии на ноль
x.var_  = x.var_.fillna(0)


In [24]:
x_means=x.copy()

### Обучим на этих признаках какой-нибдуь регрессор

In [25]:

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3,random_state=42)

In [26]:
model = RandomForestRegressor()
model.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [27]:


predictions = model.predict(X_test)

print('root_mean_squared_error = ', np.sqrt(mean_squared_error(y_test, predictions)))

root_mean_squared_error =  0.9318248704422821


In [28]:
print(model.feature_importances_)

[0.03241735 0.88140849 0.0122451  0.06009495 0.01383411]


## Попробуем на жанрах обучиться

In [29]:
x = ratings[['userId','movieId','rating']]

In [30]:
movies.head(2)

,movieId,title,genres_formatted
0,1,Toy Story (1995),adventure animation children comedy fantasy
1,2,Jumanji (1995),adventure children fantasy


In [31]:
x = pd.merge(x, movies[['movieId','genres_formatted']], on='movieId', how='left')

In [32]:
x.shape

(100836, 4)

In [33]:
x.head()

,userId,movieId,rating,genres_formatted
0,1,1,4.0,adventure animation children comedy fantasy
1,1,3,4.0,comedy romance
2,1,6,4.0,action crime thriller
3,1,47,5.0,mystery thriller
4,1,50,5.0,crime mystery thriller


In [34]:

x.isnull().any()

userId              False
movieId             False
rating              False
genres_formatted    False
dtype: bool

In [35]:
x=x.drop(['rating'], axis=1)
x=x.drop(['userId'], axis=1)
x=x.drop(['movieId'], axis=1)

In [36]:
cv_genres_ = CountVectorizer()
tf_genres = TfidfTransformer()

In [37]:
cv_genres = cv_genres_.fit_transform(x.genres_formatted)

In [38]:
cv_genres_.get_feature_names()

['action',
 'adventure',
 'animation',
 'children',
 'comedy',
 'crime',
 'documentary',
 'drama',
 'fantasy',
 'filmnoir',
 'horror',
 'imax',
 'musical',
 'mystery',
 'nogenreslisted',
 'romance',
 'scifi',
 'thriller',
 'war',
 'western']

In [39]:
len(cv_genres_.get_feature_names())

20

In [40]:
tfidf_genres= tf_genres.fit_transform(cv_genres)

In [41]:
tfidf_genres

<100836x20 sparse matrix of type '<class 'numpy.float64'>'
	with 274480 stored elements in Compressed Sparse Row format>

In [42]:

X_train, X_test, y_train, y_test = train_test_split(tfidf_genres, y, test_size=0.3,random_state=42)

In [43]:
model = RandomForestRegressor()
model.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [44]:


predictions = model.predict(X_test)

print('root_mean_squared_error = ', np.sqrt(mean_squared_error(y_test, predictions)))

root_mean_squared_error =  1.005655687573572


In [45]:
print(model.feature_importances_)

[0.10240629 0.044733   0.02658834 0.07501229 0.12313393 0.07578461
 0.00570817 0.18304634 0.04755089 0.00588055 0.0631684  0.01482789
 0.01341314 0.03203786 0.00104056 0.03644593 0.04408049 0.05099956
 0.03839732 0.01574443]


### Попробуем, нектоорые настройки CountVectorizer

In [46]:
cv_genres_mod_ = CountVectorizer(max_features=10,ngram_range=(1,3))
tf_genres_mod = TfidfTransformer()

In [47]:
cv_genres_mod = cv_genres_mod_.fit_transform(x.genres_formatted)

In [48]:
cv_genres_mod_.get_feature_names()

['action',
 'action adventure',
 'adventure',
 'comedy',
 'crime',
 'drama',
 'fantasy',
 'romance',
 'scifi',
 'thriller']

In [49]:
tfidf_genres_mod= tf_genres_mod.fit_transform(cv_genres)

In [50]:

X_train, X_test, y_train, y_test = train_test_split(tfidf_genres_mod, y, test_size=0.3,random_state=42)

In [51]:
model = RandomForestRegressor()
model.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [52]:


predictions = model.predict(X_test)

print('root_mean_squared_error = ', np.sqrt(mean_squared_error(y_test, predictions)))

root_mean_squared_error =  1.0062371739154587


#### не наблюдаем никакого улучшения, похоже

In [53]:
print(model.feature_importances_)

[0.09822683 0.051557   0.02643888 0.07993586 0.12063068 0.08166571
 0.00637514 0.17586954 0.04456899 0.00689093 0.05896096 0.0130942
 0.01271417 0.03302693 0.00035808 0.03586201 0.04780822 0.05353807
 0.03853376 0.01394404]


## Попробуем на тегах обучиться

In [54]:
tags.head()

,userId,movieId,tag_normalized
0,2,60756,funny highly quotable will ferrell
3,2,89774,boxing story mma tom hardy
6,2,106782,drugs leonardo dicaprio martin scorsese
9,7,48516,way too long
10,18,431,al pacino gangster mafia


In [55]:
x.head()

,genres_formatted
0,adventure animation children comedy fantasy
1,comedy romance
2,action crime thriller
3,mystery thriller
4,crime mystery thriller


In [56]:
x = ratings[['userId','movieId','rating']]

In [57]:
tags.head()

,userId,movieId,tag_normalized
0,2,60756,funny highly quotable will ferrell
3,2,89774,boxing story mma tom hardy
6,2,106782,drugs leonardo dicaprio martin scorsese
9,7,48516,way too long
10,18,431,al pacino gangster mafia


In [58]:
tags.shape

(1775, 3)

In [59]:
x = pd.merge(x, tags[['movieId','userId','tag_normalized']], on=['userId','movieId'], how='left')

In [60]:
x.shape

(100836, 4)

In [61]:

x.isnull().any()

userId            False
movieId           False
rating            False
tag_normalized     True
dtype: bool

In [62]:
x.tag_normalized = x.tag_normalized.fillna('')

In [63]:
x=x.drop(['rating'], axis=1)
x=x.drop(['userId'], axis=1)
x=x.drop(['movieId'], axis=1)

In [64]:
x.head()

,tag_normalized
0,
1,
2,
3,
4,


In [65]:
x.shape

(100836, 1)

In [66]:
cv_tags = CountVectorizer()
tf_tags = TfidfTransformer()

In [67]:
cv_tags = cv_tags.fit_transform(x.tag_normalized)

In [68]:
tfidf_tags= tf_tags.fit_transform(cv_genres)

In [69]:
tfidf_tags

<100836x20 sparse matrix of type '<class 'numpy.float64'>'
	with 274480 stored elements in Compressed Sparse Row format>

In [70]:

X_train, X_test, y_train, y_test = train_test_split(tfidf_tags, y, test_size=0.3,random_state=42)

In [71]:
model = RandomForestRegressor()
model.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [72]:


predictions = model.predict(X_test)

print('root_mean_squared_error = ', np.sqrt(mean_squared_error(y_test, predictions)))

root_mean_squared_error =  1.0059584080840493


In [73]:
print(model.feature_importances_)

[0.10013592 0.04814196 0.0256306  0.07155063 0.11989812 0.08261979
 0.00592953 0.18518285 0.04582321 0.00770415 0.06096368 0.01255301
 0.01328946 0.03598252 0.00035296 0.03734447 0.04738243 0.04504836
 0.04083245 0.01363388]


## Гм, а если на тех и на других призанках?

In [74]:
X_train, X_test, y_train, y_test = train_test_split(np.hstack((tfidf_tags.toarray(),tfidf_genres.toarray())), y, test_size=0.3,random_state=42)

In [75]:
model = RandomForestRegressor()
model.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [76]:


predictions = model.predict(X_test)

print('root_mean_squared_error = ', np.sqrt(mean_squared_error(y_test, predictions)))

root_mean_squared_error =  1.005707541738399


In [77]:
print(model.feature_importances_)

[0.04505027 0.02739999 0.01183469 0.04223142 0.05593191 0.02657382
 0.00418998 0.0927845  0.0215243  0.00370872 0.02902265 0.00587778
 0.00732358 0.02272109 0.00020823 0.01683142 0.02150363 0.02800851
 0.02246186 0.00503385 0.05364928 0.02468427 0.01123721 0.03912816
 0.06381354 0.04926818 0.00412971 0.08554402 0.02320257 0.00534552
 0.02504565 0.0061205  0.00758397 0.02051701 0.00043451 0.02260255
 0.02018004 0.02400021 0.01610941 0.00718146]


In [78]:
x_means.values.shape

(100836, 5)

In [79]:
y.values

array([4., 4., 4., ..., 5., 5., 3.])

### Добавим еще наши средние

In [80]:
X_train, X_test, y_train, y_test = train_test_split(np.hstack((tfidf_tags.toarray(),tfidf_genres.toarray(),x_means.values)), y.values, test_size=0.3,random_state=42)

In [81]:
model = RandomForestRegressor()
model.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [82]:


predictions = model.predict(X_test)

print('root_mean_squared_error = ', np.sqrt(mean_squared_error(y_test, predictions)))

root_mean_squared_error =  0.9457687135404536


In [83]:
print(model.feature_importances_)

[3.61346623e-03 2.86252682e-03 1.46662588e-03 1.38915725e-03
 5.09986542e-03 2.17164240e-03 5.67921665e-04 4.79652193e-03
 2.19913840e-03 2.70058759e-04 2.50322048e-03 8.27880436e-04
 9.69324120e-04 1.59676837e-03 4.68435284e-05 2.52930626e-03
 2.21131418e-03 3.40790337e-03 9.72851234e-04 6.16244380e-04
 3.54613551e-03 2.67591703e-03 1.38395917e-03 1.53575685e-03
 5.33692380e-03 2.16655545e-03 5.60606757e-04 5.34710610e-03
 1.84587287e-03 2.58143546e-04 1.99012997e-03 7.58495109e-04
 1.12201844e-03 1.60179774e-03 1.91858561e-05 2.99737652e-03
 2.61317155e-03 3.49307929e-03 1.04663821e-03 5.67673364e-04
 1.93579211e-02 8.42323491e-01 8.52521155e-03 3.84338136e-02
 1.03744387e-02]


###  И так мы тоже не превзоошли бейс-лайн

In [84]:
%%time
lr_params = {
    'fit_intercept':[False, True]
}

#fit_intercepts = [False, True]
#param_regr = dict(fit_intercept=fit_intercepts)
lr = LinearRegression()
grid_lr = GridSearchCV(lr, lr_params,
                       scoring='neg_mean_squared_error',
                       cv=3,n_jobs=-1)
grid_lr.fit(X_train, y_train)

print(grid_lr.best_params_)
print(grid_lr.best_score_)
print(grid_lr.best_estimator_)

{'fit_intercept': False}
-0.767318450578771
LinearRegression(copy_X=True, fit_intercept=False, n_jobs=None,
         normalize=False)
CPU times: user 303 ms, sys: 339 ms, total: 641 ms
Wall time: 1.46 s


In [85]:
#Оценим тчоностьна тесте
grid_lr.score(X_test,y_test)

-0.770282928393519

In [100]:

print('root_mean_squared_error = ', np.sqrt(mean_squared_error(y_test, grid_lr.best_estimator_.predict(X_test))))

root_mean_squared_error =  0.8776576373470004


In [86]:
np.sqrt(-grid_lr.score(X_test,y_test))

0.8776576373470004

In [87]:
RandomizedSearchCV

sklearn.model_selection._search.RandomizedSearchCV

In [88]:
%%time

knn_params = {
    'n_neighbors':list(range(1, 30))
   ,'weights': ['uniform', 'distance']
   ,'algorithm' : ['auto', 'ball_tree', 'kd_tree', 'brute']
   ,'leaf_size':list(range(5, 30))
}

grid_knn = RandomizedSearchCV(KNeighborsRegressor(), knn_params,  scoring='neg_mean_squared_error',
                       cv=3,n_jobs=-1)
grid_knn.fit(X_train, y_train)
print(grid_knn.best_params_)
print(grid_knn.best_score_)
print(grid_knn.best_estimator_)

{'weights': 'uniform', 'n_neighbors': 23, 'leaf_size': 26, 'algorithm': 'auto'}
-0.8259793759765974
KNeighborsRegressor(algorithm='auto', leaf_size=26, metric='minkowski',
          metric_params=None, n_jobs=None, n_neighbors=23, p=2,
          weights='uniform')
CPU times: user 429 ms, sys: 466 ms, total: 895 ms
Wall time: 4min 30s


In [89]:
grid_knn.score(X_test,y_test)

-0.8284540297657051

In [108]:

print('root_mean_squared_error = ', np.sqrt(mean_squared_error(y_test, grid_knn.best_estimator_.predict(X_test))))

root_mean_squared_error =  0.9101945010632096


In [90]:
%%time
dt_params = {
    'max_depth':[None,1,2,5,10,25,50],
     'min_samples_split':[2,5,8,10,25,50],
    
    'min_weight_fraction_leaf': [0, 0.01, 0.1, 0.15, 0.25, 0.5] ,
    'min_samples_leaf':list(range(1, 10)),
     'criterion':  ['mse', 'friedman_mse', 'mae'],
    'max_features':list(range(1, 13)) 
}
#model_forest = RandomForestRegressor(
    

grid_dt =  RandomizedSearchCV(DecisionTreeRegressor(),dt_params,  scoring='neg_mean_squared_error',
                       cv=3,n_jobs=-1)
grid_dt.fit(X_train, y_train)
print(grid_dt.best_params_)
print(grid_dt.best_score_)
print(grid_dt.best_estimator_)

{'min_weight_fraction_leaf': 0.01, 'min_samples_split': 10, 'min_samples_leaf': 9, 'max_features': 3, 'max_depth': 25, 'criterion': 'mse'}
-0.8480576268848634
DecisionTreeRegressor(criterion='mse', max_depth=25, max_features=3,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=9,
           min_samples_split=10, min_weight_fraction_leaf=0.01,
           presort=False, random_state=None, splitter='best')
CPU times: user 127 ms, sys: 31.6 ms, total: 159 ms
Wall time: 28.8 s


In [91]:

grid_dt.score(X_test,y_test)

-0.9369778535568507

In [101]:

print('root_mean_squared_error = ', np.sqrt(mean_squared_error(y_test, grid_dt.best_estimator_.predict(X_test))))

root_mean_squared_error =  0.9679761637338239


In [92]:
%%time
rf_params = {
    'n_estimators': [1,10,20,30,40,50,60,80,90],
    
    'max_depth':[None,1,2,5,10,25,50],
    
    'min_samples_leaf':list(range(1, 10)),
   
    'max_features':list(range(1, 13)) ,
     'criterion':  ['mse', 'friedman_mse', 'mae']
}

    

grid_rf =  RandomizedSearchCV(RandomForestRegressor(),rf_params,scoring='neg_mean_squared_error',
                       cv=3,n_jobs=-1)
grid_rf.fit(X_train, y_train)
print(grid_rf.best_params_)
print(grid_rf.best_score_)
print(grid_rf.best_estimator_)

{'n_estimators': 60, 'min_samples_leaf': 3, 'max_features': 7, 'max_depth': 5, 'criterion': 'friedman_mse'}
-0.8000548966482922
RandomForestRegressor(bootstrap=True, criterion='friedman_mse', max_depth=5,
           max_features=7, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=3,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=60, n_jobs=None, oob_score=False,
           random_state=None, verbose=0, warm_start=False)
CPU times: user 1.6 s, sys: 22.5 ms, total: 1.62 s
Wall time: 33min 34s


In [93]:
grid_rf.score(X_test,y_test)

-0.8010201976049784

In [102]:

print('root_mean_squared_error = ', np.sqrt(mean_squared_error(y_test, grid_rf.best_estimator_.predict(X_test))))

root_mean_squared_error =  0.8949973170937321


In [94]:

print('root_mean_squared_error = ', np.sqrt(mean_squared_error(y_test, grid_lr.best_estimator_.predict(X_test))))%%time
from sklearn.ensemble.gradient_boosting import GradientBoostingRegressor
gb_params = {
          'n_estimators': [1,10,20,30,40,50,60,80,90],
          'max_features': list(range(1, 13)),
             'max_depth':[None,1,2,5,10,25,50],
            'learning_rate': [0.1,0.3,0.5,0.7],
               #'min_samples_split':[2,5,8,10,25,50],
            'min_samples_leaf':list(range(1, 10)),
}

    

grid_gb =  RandomizedSearchCV(GradientBoostingRegressor(),gb_params,scoring='neg_mean_squared_error',
                       cv=3,n_jobs=-1)
grid_gb.fit(X_train, y_train)
print(grid_gb.best_params_)
print(grid_gb.best_score_)
print(grid_gb.best_estimator_)

{'n_estimators': 60, 'min_samples_leaf': 3, 'max_features': 8, 'max_depth': 5, 'learning_rate': 0.1}
-0.7817259124503019
GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=5, max_features=8,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=3,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=60, n_iter_no_change=None, presort='auto',
             random_state=None, subsample=1.0, tol=0.0001,
             validation_fraction=0.1, verbose=0, warm_start=False)
CPU times: user 3.29 s, sys: 83.7 ms, total: 3.37 s
Wall time: 1min 36s


In [95]:
grid_gb.score(X_test,y_test)

-0.7842340070102425

In [103]:

print('root_mean_squared_error = ', np.sqrt(mean_squared_error(y_test, grid_gb.best_estimator_.predict(X_test))))

root_mean_squared_error =  0.8855698769776682


In [96]:
%%time
SVR_params = {
          #'kernel':['linear', 'poly','rbf', 'sigmoid', 'precomputed'],
             'C' :[0.001, 0.01, 0.1, 1, 10],
    'gamma': [0.001, 0.01, 0.1, 1]
}


grid_SVR =   GridSearchCV(SVR(),SVR_params,scoring='neg_mean_squared_error',
                       cv=3,n_jobs=-1)
grid_SVR.fit(X_train, y_train)
print(grid_SVR.best_params_)
print(grid_SVR.best_score_)
print(grid_SVR.best_estimator_)

{'C': 1, 'gamma': 0.001}
-0.791743290862153
SVR(C=1, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.001,
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)
CPU times: user 5min 10s, sys: 119 ms, total: 5min 10s
Wall time: 1h 24min 52s


In [106]:

grid_SVR.score(X_test,y_test)

-0.7950338017723689

In [107]:

print('root_mean_squared_error = ', np.sqrt(mean_squared_error(y_test, grid_SVR.best_estimator_.predict(X_test))))

root_mean_squared_error =  0.8916466798975752
